# <center>Summarizer</center>

Ressources <br>
Unicode : https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string

In [1]:
# NLP Pkgs
import spacy 
import re
nlp = spacy.load('fr_core_news_sm')
# Pkgs for Normalizing Text
from spacy.lang.fr.stop_words import STOP_WORDS
from string import punctuation
# Import Heapq for Finding the Top N Sentences
from heapq import nlargest
from nltk.corpus import stopwords

stop_words = stopwords.words('french')

def summarizer_perso (path,encoding):
    raw_docx = open(path,'r',encoding=encoding)
    raw_docx = raw_docx.read().strip()
    import re
    raw_docx = re.sub('\n+',' ',raw_docx)#delete line break
    docx = nlp(raw_docx)
    # Build Word Frequency # word.text is tokenization in spacy
    word_frequencies = {}  
    for word in docx:
        if word.text not in stop_words:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1


    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    # Sentence Tokens
    sentence_list = [ sentence for sentence in docx.sents ]

    # Sentence Scores
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]


    summarized_sentences = nlargest(3, sentence_scores, key=sentence_scores.get)
    final_sentences = [ w.text for w in summarized_sentences ]
    summary = ' '.join(final_sentences)
    return summary

In [2]:
summarizer_perso('resume_pluies.txt','utf-8')

"Partout, les eaux avaient monté, les rivières enflées, on consolidait les digues, on en bâtissait d'autres, plus hautes, mais qui se révélaient de nouveau insuffisantes. Le phénomène, inexplicable, échappait à toute logique, à toute prévision, à tout modèle, à toute saison.   Sur le port, les réfugiés se battent pour prendre place dans les derniers bateaux, pris de panique, convaincus qu'il s’agit là du dernier espoir de s'en sortir."

In [3]:
summarizer_perso('resume_cuisine-jap.txt','utf-8-sig')

"Des recettes, des anecdotes, des histoires, des cartes, des gestes, des techniques, des ustensiles, des ingrédients… En un coup d’œil, toutes les réponses aux questions que l'on se pose ! Un livre entièrement illustré pour tout savoir sur la cuisine japonaise !"

In [4]:
summarizer_perso('resume_prix_clara.txt','utf-8')

"Amour, science-fiction, polar, témoignage, aventure : tous les genres sont explorés par ces jeunes avec brio, révélant ainsi leur intérêt et leur talent pour l'écriture. Ces textes surprennent par leur fraîcheur, leur originalité, leur sincérité, et forment un kaléidoscope de l'imaginaire adolescent. Ce prix a été créé en mémoire de Clara, décédée subitement à l’âge de 13 ans des suites d’une malformation cardiaque."

## Sumy function 

In [5]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
def summarizer_sumy (path,encoding):
	docx = open(path,'r',encoding=encoding)
	docx = docx.read()
	parser = PlaintextParser.from_string(docx,Tokenizer("french"))
	lex_summarizer = LexRankSummarizer()
	summary = lex_summarizer(parser.document,3)
	summary_list = [str(sentence) for sentence in summary]
	result = ' '.join(summary_list)
	return result

In [6]:
summarizer_sumy('resume_cuisine-jap.txt','utf-8')

'\ufeffUn livre entièrement illustré pour tout savoir sur la cuisine japonaise ! Des recettes, des anecdotes, des histoires, des cartes, des gestes, des techniques, des ustensiles, des ingrédients… Une invitation à la découverte des coutumes et des saveurs japonaises !Comment façonner les sushi ? Quelle est la recette traditionnelle de la soupe miso ?'